## Este notebook tem por motivação apenas demonstrar como usar cross-validation + GridSearch com a biblioteca xgboost
Explicações mais detalhadas acerca de *encoding* dos dados e mais, acesse [o link][main].
Para explicações acerca do funcionamento do **xgboost**, acesse [aqui.][]

[main]: main.ipynb
[teorico]: xgb.ipynb

In [88]:
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn

import category_encoders as ce
import matplotlib.pyplot as plt
import matplotlib as mplt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid

from sklearn.model_selection import StratifiedKFold


from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

from time import time
from sklearn.metrics import plot_confusion_matrix

def dataframe_encoder(data, encoder=LabelEncoder()):
    _ = data.copy()
    for c in _.columns:
        v = pd.Series(data[c].values, name=c, index=data.index)    
        if v.dtype == 'object':
            _[c] = encoder.fit_transform(_[c])
    return _

Como de praxe, carregamos os dados e dividimo-los em conjuntos de treinamento e teste. Além disso, *X* e *y* são os dados já mapeados para numéricos apenas.

In [71]:
train = pd.read_csv('train.csv').drop('id', axis=1)
test = pd.read_csv('test.csv').drop('id', axis=1)

X, y = dataframe_encoder(train.iloc[:, :-1]), train.iloc[:, -1]
Xy = dataframe_encoder(train)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.2, random_state=42
)

# CrossValidation + GridSearch: API nativa
Primeiro convertemos os dados para o formato esperado pela API nativa, depois são definidos os parâmetros a serem testados pelo GridSearch.

In [109]:
dtrain = xgb.DMatrix(
    X,
    label=y
)
param_opts = {
    'max_depth':[2, 3, 4],
    'eta':[.3,],
    'gamma': [.5, 1],
    'colsample_bytree': np.array([1.0,]),
    'subsample': [1.0,],    
}

In [110]:
param_list = list(ParameterGrid(param_opts))

In [112]:
%%time
for param in params_list:
    model = xgb.cv(
        param,
        dtrain,
        num_boost_round=100,
        nfold=3,
        stratified=True,
        seed=42,
        as_pandas=False,
        shuffle=False
    )
    

CPU times: user 4min 29s, sys: 420 ms, total: 4min 29s
Wall time: 4min 29s


# CrossValidation + GridSearch: "the sklearn-way"

In [107]:
model = xgb.XGBRegressor(
    n_jobs=-1,
    random_state=42,    
)
param_opts = {
    'max_depth':[2, 3, 4],
    'eta':[.3,],
    'gamma': [.5, 1],
    'colsample_bytree': np.array([1.0,]),
    'subsample': [1.0,],
}

In [108]:
%%time

clf = GridSearchCV(
    model,
    param_opts,
    cv=3,
    verbose=0,
    n_jobs=-1,
);
clf.fit(
    X, y
);

CPU times: user 27.1 s, sys: 488 ms, total: 27.6 s
Wall time: 2min 18s


GridSearchCV(cv=3, error_score=nan,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estima...
                                    random_state=42, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                                    subsample=None, tree_method=None,
                                    validate_parameters=False, verbosity=None

A vantagem de usar tal abordagem é que ela segue um padrão bem definido e seguido pelo sklearn, não necessitando portanto de um conhecimento aespefícico acerca do funcionamento do *xgboost* para sua plena utilização.

### Para ver quais os parâmetros do melhor classificador, basta descomentar e executar a célula abaixo.

In [113]:
# clf.best_estimator_

In [87]:
%%time
best_param = {
    'max_depth':3,
    'eta':1,
}
model = xgb.XGBClassifier(
    max_depth=3,
    learning_rate=1,    
    n_estimators=100,
)
model.fit(X_train, y_train);

CPU times: user 15.8 s, sys: 24 ms, total: 15.8 s
Wall time: 15.9 s


XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [121]:
%%time
pred = (clf.predict(X_test) >= .3).astype(np.int8)
print('acc:', accuracy_score(y_true=y_test, y_pred=pred))
print('f1:', f1_score(y_true=y_test, y_pred=pred))
print("Matriz de confusão:\n", confusion_matrix(y_test, pred, normalize='all'))

acc: 0.68255
f1: 0.5965729777815432
Matriz de confusão:
 [[0.44783333 0.24678333]
 [0.07066667 0.23471667]]
CPU times: user 344 ms, sys: 2 µs, total: 344 ms
Wall time: 343 ms
